<a href="https://colab.research.google.com/github/shashank3110/fine_tuning_models/blob/main/fine_tune_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -U datasets huggingface_hub fsspec


In [ ]:
%pip install evaluate

In [ ]:
#dataset download issue fix: https://discuss.huggingface.co/t/valueerror-invalid-pattern-can-only-be-an-entire-path-component/80450

In [3]:
from huggingface_hub import login

login()

In [4]:
from datasets import load_dataset
from transformers import AutoTokenizer

## Load dataset and Tokenizer

In [ ]:
dataset = load_dataset("yelp_review_full", )
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

In [7]:
# Tokenization function
def tokenize(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)



In [8]:
dataset.shape

{'train': (650000, 2), 'test': (50000, 2)}

In [9]:
# tokenize full dataset
# tokenized_dataset = dataset.map(tokenize, batched=True)

In [10]:
# tokenized_dataset

In [11]:
# train - test subset to train on smaller sample before doing full finetuning
# train_subset = tokenized_dataset["train"].shuffle(seed=42).select(range(1000))
# test_subset = tokenized_dataset["test"].shuffle(seed=42).select(range(1000))

train_subset = dataset["train"].shuffle(seed=42).select(range(1000))
test_subset =  dataset["test"].shuffle(seed=42).select(range(1000))

In [ ]:
# tokenize subset
train_subset = train_subset.map(tokenize, batched=True)
test_subset = test_subset.map(tokenize, batched=True)

## Load Bert model

In [13]:
from transformers import AutoModelForSequenceClassification

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=5)

In [15]:
print(model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [21]:
# !printenv

## eval function

In [22]:
import numpy as np
import evaluate

In [23]:
metric = evaluate.load("accuracy")

In [24]:
def eval_model(eval_preds):
  logits, labels = eval_preds
  preds = np.argmax(logits, axis=1)
  return metric.compute(predictions=preds, references=labels)

## Create Trainer





In [25]:
from transformers import TrainingArguments, Trainer


In [26]:
# training args
training_args = TrainingArguments(
    output_dir="yelp_review_classifier",
    eval_strategy="epoch",
    push_to_hub=True,
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
)

In [27]:
# train and eval on a small subset
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_subset,
    eval_dataset=test_subset,
    compute_metrics=eval_model,
)


In [32]:
%%time
train_output = trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.107062,0.545000
2,No log,1.036452,0.572000


CPU times: user 3min 57s, sys: 3.39 s, total: 4min
Wall time: 4min 14s


In [33]:
# print metrics
print(train_output.metrics)

{'train_runtime': 254.4236, 'train_samples_per_second': 7.861, 'train_steps_per_second': 0.252, 'total_flos': 526236284928000.0, 'train_loss': 1.0729267597198486, 'epoch': 2.0}


In [36]:
# dir(train_output)

In [29]:
# trainer.push_to_hub()


In [38]:
# train_subset['label']